In [3]:
import geopandas as gpd
import pickle
from pprint import pprint

In [4]:
base_path = '../../Documents/Research/Guatemala/guat_obia/'
training_data = base_path + 'segs/training_segments.shp'
src = gpd.read_file(training_data)

In [5]:
src.shape

(574031, 22)

In [6]:
src

,DN,count,perimeter,eccentrici,equal_diam,major_axis,minor_axis,orientatio,red_mean,green_mean,blue_mean,sobel_mean,sobel_sum,sobel_std,class_id,training,testing,class,photo,best_guess,random,geometry
0,21,25,16.828427,0.730195,5.641896,6.937923,4.740256,-1.223299,118.320000,144.480000,90.520000,0.117887,2.947163,0.120952,0,0,0,None,new_IMG_3451,None,0.172,"POLYGON ((476762.2364276428 1952786.041789225,..."
1,32,3,1.000000,1.000000,1.954410,3.265986,0.000000,-0.000000,127.000000,134.000000,90.000000,0.362157,1.086471,0.008216,0,0,0,None,new_IMG_3451,None,0.502,"POLYGON ((476792.0941418455 1952785.722385633,..."
2,33,3,1.000000,1.000000,1.954410,3.265986,0.000000,-0.000000,120.000000,131.000000,86.666667,0.342942,1.028827,0.003410,0,0,0,None,new_IMG_3451,None,0.180,"POLYGON ((476792.732809529 1952785.722385633, ..."
3,39,1,0.000000,0.000000,1.128379,0.000000,0.000000,0.785398,165.000000,172.000000,103.000000,0.475357,0.475357,0.000000,0,0,0,None,new_IMG_3451,None,0.491,"POLYGON ((476806.7834985655 1952785.562683837,..."
4,41,20,16.242641,0.920775,5.046265,8.068109,3.147319,0.049366,209.050000,213.050000,130.700000,0.260045,5.200909,0.256232,0,0,0,None,new_IMG_3451,None,0.973,"POLYGON ((476811.4138392708 1952785.402982041,..."
5,43,1,0.000000,0.000000,1.128379,0.000000,0.000000,0.785398,126.000000,143.000000,107.000000,0.350121,0.350121,0.000000,0,0,0,None,new_IMG_3451,None,0.439,"POLYGON ((476814.9265115299 1952785.402982041,..."
6,46,12,11.000000,0.966702,3.908820,7.406234,1.895294,0.076160,106.833333,114.250000,84.666667,0.200514,2.406173,0.130844,0,0,0,None,new_IMG_3451,None,0.325,"POLYGON ((476820.1955199186 1952785.402982041,..."
7,57,1,0.000000,0.000000,1.128379,0.000000,0.000000,0.785398,253.000000,251.000000,217.000000,0.517737,0.517737,0.000000,0,0,0,None,new_IMG_3451,None,0.250,"POLYGON ((476845.2632264952 1952785.083578449,..."
8,1,277,65.071068,0.556187,18.779972,21.141327,17.569646,0.869276,128.039711,146.902527,93.140794,0.072170,19.991008,0.094190,0,0,0,None,new_IMG_3451,None,0.916,"POLYGON ((476706.1933384174 1952786.840298204,..."
9,8,139,46.520815,0.619938,13.303394,15.593643,12.235565,0.298167,145.503597,162.172662,92.050360,0.079692,11.077137,0.119558,0,0,0,None,new_IMG_3451,None,0.534,"POLYGON ((476726.311370447 1952786.520894612, ..."


In [7]:
training_fields = ['count', 'perimeter', 'eccentrici', 'equal_diam', 'major_axis',
          'minor_axis', 'orientatio', 'red_mean', 'green_mean',
          'blue_mean', 'sobel_mean', 'sobel_sum', 'sobel_std']
class_id_field = ['class_id']

In [20]:
'''
training_fraction = 0.50
s = src.query(class_id_field[0] + ' != 0')
s1 = s.sample(frac=training_fraction).sort_index()
s2 = s[~s.isin(s1)].dropna(subset=[fields[0]])
selection1 = s1[fields]
selection2 = s2[fields]
'''

training_fraction = 0.250 # Percentage of objects in the training set
s = src.query(class_id_field[0] + ' != 0') # Select objects with a class id
t = s.sample(frac=training_fraction).sort_index() # Training objects
v = s[~s.isin(t)].dropna(subset=[training_fields[0]]) # Verification objects
training_X = t[training_fields] # Training objects with correct fields
training_Y = t[class_id_field].values.reshape(-1) # Training objects with correct class id field
verification_X = v[training_fields] # Verification objects with correct fields
verification_Y = v[class_id_field].values.reshape(-1) # Verification objects with correct class id field

print("Total size: " + str(s.shape) + "\n" \
      "Training_X: " + str(training_X.shape) + ", Verification_X: " + str(verification_X.shape) + "\n" \
      "Training_Y: " + str(training_Y.shape) + ", Verification_Y: " + str(verification_Y.shape))

Total size: (91963, 22)
Training_X: (22991, 13), Verification_X: (68972, 13)
Training_Y: (22991,), Verification_Y: (68972,)


In [11]:
training_X

,count,perimeter,eccentrici,equal_diam,major_axis,minor_axis,orientatio,red_mean,green_mean,blue_mean,sobel_mean,sobel_sum,sobel_std
1565,263,75.183766,0.563232,18.299235,21.539132,17.797754,0.163271,209.969582,209.171103,160.448669,0.033595,8.835384,0.015777
1572,342,90.597980,0.415433,20.867389,23.130252,21.039822,0.356577,205.298246,210.201754,150.789474,0.034733,11.878515,0.020131
2010,383,89.012193,0.733460,22.082816,28.066129,19.077454,0.756110,165.107050,174.971279,131.676240,0.045106,17.275703,0.028362
2014,292,86.734019,0.622224,19.281752,22.963043,17.976367,0.698552,212.071918,213.482877,157.195205,0.042840,12.509220,0.024376
2056,313,75.597980,0.804233,19.963065,26.787189,15.920005,1.294493,218.645367,211.648562,173.444089,0.031230,9.774875,0.014195
2141,404,86.597980,0.410907,22.680141,24.412977,22.256752,-1.064474,189.896040,199.037129,142.051980,0.048725,19.684700,0.026292
2143,326,74.012193,0.525034,20.373416,22.280612,18.962616,1.060731,203.319018,213.263804,146.825153,0.031622,10.308655,0.014782
2144,307,71.597980,0.439063,19.770800,21.824330,19.608211,-0.714487,203.114007,207.859935,149.706840,0.033892,10.404864,0.018759
2240,408,81.597980,0.522800,22.792142,24.848761,21.182453,1.223180,190.924020,207.338235,134.757353,0.040941,16.703799,0.023557
2257,306,82.426407,0.704070,19.738574,24.459446,17.369390,1.423189,192.869281,205.578431,145.307190,0.036280,11.101738,0.017778


In [25]:
verification_X

,count,perimeter,eccentrici,equal_diam,major_axis,minor_axis,orientatio,red_mean,green_mean,blue_mean,sobel_mean,sobel_sum,sobel_std
1512,304.0,74.183766,0.480577,19.673963,21.722933,19.049984,-0.815351,194.976974,202.532895,145.009868,0.036520,11.101970,0.016640
1565,263.0,75.183766,0.563232,18.299235,21.539132,17.797754,0.163271,209.969582,209.171103,160.448669,0.033595,8.835384,0.015777
1572,342.0,90.597980,0.415433,20.867389,23.130252,21.039822,0.356577,205.298246,210.201754,150.789474,0.034733,11.878515,0.020131
1585,349.0,78.426407,0.591362,21.079862,24.128016,19.456976,-1.013900,204.455587,214.008596,149.573066,0.034884,12.174373,0.017893
1608,386.0,88.976659,0.587228,22.169133,25.343920,20.513910,0.620566,187.443005,194.665803,144.227979,0.039332,15.182289,0.020740
1612,391.0,81.012193,0.698488,22.312254,27.095770,19.390324,-0.256351,204.053708,213.122762,146.132992,0.034056,13.315758,0.018556
1617,200.0,60.769553,0.759510,15.957691,20.682004,13.453561,0.962487,210.645000,209.075000,163.005000,0.025270,5.054099,0.016117
1655,339.0,76.769553,0.457772,20.775664,22.507013,20.010299,-1.087491,198.902655,207.961652,146.079646,0.032384,10.978236,0.018416
1660,389.0,83.976659,0.600835,22.255116,25.417162,20.317800,0.787671,168.596401,185.753213,120.473008,0.037783,14.697557,0.020910
1677,361.0,77.597980,0.363494,21.439204,22.918342,21.350649,1.506366,192.819945,207.177285,136.709141,0.036922,13.328837,0.021971


In [26]:
training_Y

array([1, 1, 1, ..., 9, 9, 9])

In [27]:
verification_Y

array([1., 1., 1., ..., 9., 9., 9.])

In [29]:
import pandas as pd
pd.concat([training_X, verification_X])

,count,perimeter,eccentrici,equal_diam,major_axis,minor_axis,orientatio,red_mean,green_mean,blue_mean,sobel_mean,sobel_sum,sobel_std
1513,284.0,74.319805,0.622639,19.015784,22.084039,17.280968,0.417377,212.024648,209.190141,161.330986,0.028251,8.023276,0.015572
1638,335.0,88.083261,0.565853,20.652730,24.834246,20.475981,-1.082960,185.322388,189.940299,145.982090,0.032059,10.739779,0.018212
1737,321.0,89.012193,0.613155,20.216575,24.793563,19.585993,0.581137,145.464174,164.485981,113.781931,0.036352,11.669125,0.021807
2010,383.0,89.012193,0.733460,22.082816,28.066129,19.077454,0.756110,165.107050,174.971279,131.676240,0.045106,17.275703,0.028362
2056,313.0,75.597980,0.804233,19.963065,26.787189,15.920005,1.294493,218.645367,211.648562,173.444089,0.031230,9.774875,0.014195
2121,371.0,77.769553,0.644253,21.734118,25.194769,19.269265,0.208794,205.708895,206.293801,158.450135,0.044779,16.613011,0.026535
2141,404.0,86.597980,0.410907,22.680141,24.412977,22.256752,-1.064474,189.896040,199.037129,142.051980,0.048725,19.684700,0.026292
2144,307.0,71.597980,0.439063,19.770800,21.824330,19.608211,-0.714487,203.114007,207.859935,149.706840,0.033892,10.404864,0.018759
2166,258.0,65.112698,0.656227,18.124453,21.277467,16.055194,1.232730,214.007752,222.034884,159.891473,0.024777,6.392532,0.013595
2172,263.0,75.355339,0.751197,18.299235,23.665060,15.620796,-1.280967,203.403042,211.851711,150.349810,0.040711,10.706934,0.017455


In [30]:
type(src)

geopandas.geodataframe.GeoDataFrame

In [31]:
pd.concat([src, src])

,DN,count,perimeter,eccentrici,equal_diam,major_axis,minor_axis,orientatio,red_mean,green_mean,blue_mean,sobel_mean,sobel_sum,sobel_std,class_id,training,testing,class,photo,best_guess,random,geometry
0,21,25,16.828427,0.730195,5.641896,6.937923,4.740256,-1.223299,118.320000,144.480000,90.520000,0.117887,2.947163,0.120952,0,0,0,None,new_IMG_3451,None,0.172,"POLYGON ((476762.2364276428 1952786.041789225,..."
1,32,3,1.000000,1.000000,1.954410,3.265986,0.000000,-0.000000,127.000000,134.000000,90.000000,0.362157,1.086471,0.008216,0,0,0,None,new_IMG_3451,None,0.502,"POLYGON ((476792.0941418455 1952785.722385633,..."
2,33,3,1.000000,1.000000,1.954410,3.265986,0.000000,-0.000000,120.000000,131.000000,86.666667,0.342942,1.028827,0.003410,0,0,0,None,new_IMG_3451,None,0.180,"POLYGON ((476792.732809529 1952785.722385633, ..."
3,39,1,0.000000,0.000000,1.128379,0.000000,0.000000,0.785398,165.000000,172.000000,103.000000,0.475357,0.475357,0.000000,0,0,0,None,new_IMG_3451,None,0.491,"POLYGON ((476806.7834985655 1952785.562683837,..."
4,41,20,16.242641,0.920775,5.046265,8.068109,3.147319,0.049366,209.050000,213.050000,130.700000,0.260045,5.200909,0.256232,0,0,0,None,new_IMG_3451,None,0.973,"POLYGON ((476811.4138392708 1952785.402982041,..."
5,43,1,0.000000,0.000000,1.128379,0.000000,0.000000,0.785398,126.000000,143.000000,107.000000,0.350121,0.350121,0.000000,0,0,0,None,new_IMG_3451,None,0.439,"POLYGON ((476814.9265115299 1952785.402982041,..."
6,46,12,11.000000,0.966702,3.908820,7.406234,1.895294,0.076160,106.833333,114.250000,84.666667,0.200514,2.406173,0.130844,0,0,0,None,new_IMG_3451,None,0.325,"POLYGON ((476820.1955199186 1952785.402982041,..."
7,57,1,0.000000,0.000000,1.128379,0.000000,0.000000,0.785398,253.000000,251.000000,217.000000,0.517737,0.517737,0.000000,0,0,0,None,new_IMG_3451,None,0.250,"POLYGON ((476845.2632264952 1952785.083578449,..."
8,1,277,65.071068,0.556187,18.779972,21.141327,17.569646,0.869276,128.039711,146.902527,93.140794,0.072170,19.991008,0.094190,0,0,0,None,new_IMG_3451,None,0.916,"POLYGON ((476706.1933384174 1952786.840298204,..."
9,8,139,46.520815,0.619938,13.303394,15.593643,12.235565,0.298167,145.503597,162.172662,92.050360,0.079692,11.077137,0.119558,0,0,0,None,new_IMG_3451,None,0.534,"POLYGON ((476726.311370447 1952786.520894612, ..."
